Expresso

In [1]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError


def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['ctt2025']
    return db


def criar_colecao(nome_colecao):
    db = conecta_db()
    if nome_colecao not in db.list_collection_names():
        db.create_collection(nome_colecao)
        print(f"Coleção '{nome_colecao}' criada.")
    else:
        print(f"Coleção '{nome_colecao}' já existe.")


def inserir_documento(nome_colecao, documento):
    db = conecta_db()
    colecao = db[nome_colecao]
    try:
        colecao.insert_one(documento)
        print("Documento inserido com sucesso.")
    except PyMongoError as error:
        print("Erro ao inserir documento:", error)


# Exemplo de uso
criar_colecao("expresso_2025")

# Documento de exemplo (substitui pelos dados reais)
'''
documento_exemplo = {
    "DATA_CRIACAO": "2025-04-24",
    "CENTRO": "Lisboa",
    "Giro": "G123",
    "LOPTICA": "LO1",
    "JANELA_HORARIA": "08:00-10:00",
    "NOME": "João Silva",
    "MORADA": "Rua das Flores, 123",
    "CP": "1000-000",
    "LOCALIDADE": "Lisboa",
    "COD_T_EVEN": "EVT456",
    "DATA_EVENTO": "2025-04-23",
    "LATITUDE": "38.7169",
    "LONGITUDE": "-9.1399",
    "NOME_REM": "Maria Silva",
    "COD_PAIS_ORIGEM": "PT"
}

#inserir_documento("expresso_2025", documento_exemplo)
'''

Coleção 'expresso_2025' já existe.


'\ndocumento_exemplo = {\n    "DATA_CRIACAO": "2025-04-24",\n    "CENTRO": "Lisboa",\n    "Giro": "G123",\n    "LOPTICA": "LO1",\n    "JANELA_HORARIA": "08:00-10:00",\n    "NOME": "João Silva",\n    "MORADA": "Rua das Flores, 123",\n    "CP": "1000-000",\n    "LOCALIDADE": "Lisboa",\n    "COD_T_EVEN": "EVT456",\n    "DATA_EVENTO": "2025-04-23",\n    "LATITUDE": "38.7169",\n    "LONGITUDE": "-9.1399",\n    "NOME_REM": "Maria Silva",\n    "COD_PAIS_ORIGEM": "PT"\n}\n\n#inserir_documento("expresso_2025", documento_exemplo)\n'

In [2]:
from pymongo import MongoClient
import numpy as np
import re
import datetime  # <<< ADIÇÃO: para detetar datetime/date

def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Conectar à base de dados
db = conecta_db()
colecao = db['expresso_2025']

# Obter todas as datas distintas do campo DATA_CRIACAO
datas = colecao.distinct("data_criacao")

data_validation = []

# Expressão regular para datas no formato DD/MM/YYYY (legado)
#regex_data = re.compile(r"^(\d{4})\.(\d{1,2})\.(\d{1,2})$")
#regex_data = re.compile(r"^(\d{4})-(\d{1,2})-(\d{1,2})$")
regex_data = re.compile(r"^(\d{2})/(\d{2})/(\d{4})$")

for data in datas:
    if isinstance(data, (datetime.datetime, datetime.date)):  # <<< ADIÇÃO
        y = data.year
        m = f"{data.month:02d}"
        d = f"{data.day:02d}"
        data_validation.append(f"{y}{m}{d}")
    elif isinstance(data, str):
        match = regex_data.match(data.strip())
        if match:
            dia, mes, ano = match.groups()
            data_formatada = ano + mes.zfill(2) + dia.zfill(2)
            data_validation.append(data_formatada)
        else:
            print(f"Formato inválido ignorado: {data}")
    else:
        # tipos inesperados (None, etc.)
        print(f"Tipo de data não suportado ignorado: {type(data)} -> {data}")

# Eliminar duplicados, ordenar e converter para numpy array
data_validation = np.array(sorted(set(data_validation)))
print(data_validation)


['20250823' '20250824' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250830' '20250831' '20250901' '20250902' '20250903'
 '20250904' '20250905' '20250906' '20250907' '20250908' '20250909'
 '20250910' '20250911' '20250912' '20250913' '20250914' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250920' '20250921'
 '20250922' '20250923' '20250924' '20250925' '20250926' '20250927'
 '20250928' '20250929' '20250930' '20251001' '20251002']


In [ ]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
from pymongo import MongoClient
import time
import re

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Verifica datas já existentes na coleção REDE_BASE_2025
def obter_datas_existentes():
    db = conecta_db()
    colecao = db["expresso_2025"]
    datas = colecao.distinct("data_criacao")

    regex_data = re.compile(r"^(\d{2})/(\d{2})/(\d{4})$")

    datas_formatadas = []

    for data in datas:
        # >>> ALTERAÇÃO (suporta datetime e legado string DD/MM/YYYY)
        if isinstance(data, (datetime.date, datetime.datetime)):
            y = data.year
            m = f"{data.month:02d}"
            d = f"{data.day:02d}"
            datas_formatadas.append(f"{y}{m}{d}")
        elif isinstance(data, str):
            match = regex_data.match(data.strip())
            if match:
                dia, mes, ano = match.groups()
                data_formatada = ano + mes.zfill(2) + dia.zfill(2)
                datas_formatadas.append(data_formatada)
        # <<<

    return set(datas_formatadas)

# Processa linha
def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError:
        decoded_line = row.decode('utf-8')

    if len(decoded_line.split(';')) == 13:
        return decoded_line.strip()
    else:
        print(f"Ignorando linha com colunas incorretas: {decoded_line}")
        return None

# Ligar FTP e obter ficheiros
ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')
ftp.cwd('/OP/Rotas/EventosPDA')
files = ftp.nlst()
ftp.quit()

# Obter datas validadas
data_validation = obter_datas_existentes()

# Processar ficheiros
for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        ano = file[14:18]
        mes = file[18:20]

        if date_str_s not in data_validation and ano == "2025" and mes >= "01":
            print(f"A processar: {date_str_s}")

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)

            try:
                df = pd.read_csv(file_data, sep=';', on_bad_lines='skip', encoding="Latin-1")
            except pd.errors.ParserError as pe:
                print(f"Erro ao ler CSV no ficheiro '{file}': {pe}")
                ftp.quit()
                continue

            ftp.quit()

            if df.empty:
                print(f"Ficheiro '{file}' vazio ou com linhas inválidas. Ignorado.")
                continue

            # >>> ALTERAÇÃO: nomes de colunas em minúsculas
            df.columns = [c.lower() for c in df.columns]
            # <<<

            # Mantém limpeza de nulos como estava
            df = df.where(pd.notna(df), None)

            # >>> ALTERAÇÃO: data_criacao como datetime (Date no Mongo)
            dia_i = int(date_str_s[6:8])
            mes_i = int(mes)
            ano_i = int(ano)
            data_dt = datetime.datetime(ano_i, mes_i, dia_i)
            df["data_criacao"] = data_dt
            # <<<

            # >>> ALTERAÇÃO: restantes campos como string
            for col in df.columns:
                if col != "data_criacao":
                    df[col] = df[col].astype(str)
                    df[col] = df[col].replace(["nan", "None"], "")
            # <<<

            db = conecta_db()
            colecao = db['expresso_2025']

            try:
                documentos = df.to_dict(orient='records')
                if documentos:
                    colecao.insert_many(documentos)
                    print(f"Ficheiro '{file}' inserido com sucesso ({len(documentos)} registos). Tempo: {time.time() - start_time:.2f}s")
            except Exception as e:
                print(f"Erro ao inserir dados do ficheiro '{file}': {e}")
        else:
            print(f"Ficheiro '{file}' ignorado (já inserido ou data inválida).")


Ficheiro 'EVPDA3466_ENV_20250906_20250907045626.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250907_20250908010515.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250908_20250909010504.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250909_20250910010505.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250910_20250911010508.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250911_20250912010502.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250912_20250913010520.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250913_20250914010502.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250914_20250915010510.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250915_20250916010528.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVPDA3466_ENV_20250916_20250917010521.CSV' ignorad

Banca

In [ ]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['ctt2025']
    return db

# Criar coleção (opcional em MongoDB)
def criar_colecao(nome_colecao):
    db = conecta_db()
    if nome_colecao not in db.list_collection_names():
        db.create_collection(nome_colecao)
        print(f"Coleção '{nome_colecao}' criada.")
    else:
        print(f"Coleção '{nome_colecao}' já existe.")

# Inserir um documento (linha)
def inserir_documento(nome_colecao, documento):
    db = conecta_db()
    colecao = db[nome_colecao]
    try:
        colecao.insert_one(documento)
        print("Documento inserido com sucesso.")
    except PyMongoError as error:
        print(f"Erro ao inserir documento: {error}")

# Criar a coleção (equivalente a "CREATE TABLE IF NOT EXISTS")
criar_colecao("banca_2025")

# Exemplo de inserção (podes adaptar à tua fonte de dados)
documento_exemplo = {
    "COD_CLIENTE_BNC": "123456",
    "CODIGO_BALCAO_SIDIR": "001",
    "CODIGO_BALCAO_AF": "AF456",
    "AGENCIA": "Lisboa",
    "CA": "Central",
    "CIRCUITO": "C1",
    "DATA": "24/04/2025",
    "CONTACTO": "João Silva",
    "FATURAR": "Sim",
    "ENTREGUE": "Não",
    "TAREFA": "Entrega",
    "CP": "1000-001",
    "CENTRO_OPERACIONAL": "Centro A",
    "MORADA_TOQUE": "Rua das Flores",
    "HORA_TOQUE_PREVISTA": "10:00",
    "HORA_TOQUE_REAL": "10:15"
}

# Inserir esse documento na coleção
#inserir_documento("banca_2025", documento_exemplo)


Coleção 'banca_2025' já existe.


In [ ]:
from pymongo import MongoClient
import numpy as np
import re

def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Conectar à base de dados
db = conecta_db()
colecao = db['banca_2025']

# Obter todas as datas distintas do campo DATA
datas = colecao.distinct("data")

data_validation = []

# Expressão regular correta: YYYY.MM.DD
regex_data = re.compile(r"^(\d{4})\.(\d{1,2})\.(\d{1,2})$")

for data in datas:
    if isinstance(data, str):
        match = regex_data.match(data.strip())
        if match:
            ano, mes, dia = match.groups()
            data_formatada = ano + mes.zfill(2) + dia.zfill(2)
            data_validation.append(data_formatada)
        else:
            print(f"Formato inválido ignorado: {data}")

# Eliminar duplicados, ordenar e converter para numpy array
data_validation = np.array(sorted(set(data_validation)))
print(data_validation)


[]


In [ ]:
# Conexão FTP
ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')
ftp.cwd('/OP/Rotas/Banca_Recolhas')
files = ftp.nlst()
ftp.quit()

# Datas já presentes
data_validation = obter_datas_existentes()

# Processamento
for file in files:
    start_time = time.time()
    if file.startswith("C"):
        matches = re.findall(r"\d{8}", file)
        if matches:
            date_str_s = matches[0]  # usa a primeira data no nome
            ano = date_str_s[:4]
        else:
            print(f"⚠️ Nome inválido sem data no ficheiro: {file}")
            continue

        #print(f"→ A verificar ficheiro: {file}")
        #print(f"  Data extraída: {date_str_s}")
        #print(f"  Está em data_validation? {'SIM' if date_str_s in data_validation else 'NÃO'}")

        if date_str_s not in data_validation and ano == "2025":
            # Reabrir conexão FTP para o ficheiro
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)
            lines = file_data.readlines()
            ftp.quit()

            def process_data(row):
                try:
                    return row.decode('Latin-1').strip()
                except UnicodeDecodeError:
                    return row.decode('utf-8').strip()

            data_rows = [process_data(row) for row in lines[:-1]]

            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]

            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')
            decoded_data = csv_data.decode('utf-8-sig')

            if decoded_data.strip():
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', on_bad_lines='skip', encoding="Latin-1")

                column_mapping = {
                    'ï»¿COD_CLIENTE_BNC': 'COD_CLIENTE_BNC',
                    'CODIGO_BALCAO_SIDIR': 'CODIGO_BALCAO_SIDIR',
                    'CODIGO_BALCAO_AF': 'CODIGO_BALCAO_AF',
                    'AGENCIA': 'AGENCIA',
                    'CA': 'CA',
                    'CIRCUITO': 'CIRCUITO',
                    'DATA': 'DATA',
                    'CONTACTO': 'CONTACTO',
                    'FATURAR': 'FATURAR',
                    'ENTREGUE': 'ENTREGUE',
                    'TAREFA': 'TAREFA',
                    'CP': 'CP',
                    'CENTRO OPERACIONAL': 'CENTRO_OPERACIONAL',
                    'MORADA TOQUE': 'MORADA_TOQUE',
                    'HORA TOQUE PREVISTA': 'HORA_TOQUE_PREVISTA',
                    'HORA TOQUE REAL': 'HORA_TOQUE_REAL'
                }

                df.rename(columns=column_mapping, inplace=True)
                df = df.where(pd.notna(df), None)

                # Forçar DATA no formato YYYY.MM.DD
                data_formatada = f"{date_str_s[:4]}.{date_str_s[4:6]}.{date_str_s[6:]}"
                df['data'] = data_formatada
                print(f"→ Será inserida a data: {data_formatada}")

                db = conecta_db()
                colecao = db['banca_2025']

                try:
                    documentos = df.to_dict(orient='records')
                    if documentos:
                        colecao.insert_many(documentos)
                        print(f"✅ Ficheiro '{file}' inserido com sucesso ({len(documentos)} registos). Tempo: {time.time() - start_time:.2f}s")
                except Exception as e:
                    print(f"❌ Erro ao inserir dados do ficheiro '{file}': {e}")
        else:
            print(f"IGNORADO: {file} (data: {date_str_s}) — já inserido ou fora do ano alvo.")


IGNORADO: CAX019_TOQUE_BANCA_D_20250903_20250904000626.CSV (data: 20250903) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250904_20250905000614.CSV (data: 20250904) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250905_20250906000607.CSV (data: 20250905) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250908_20250909000621.CSV (data: 20250908) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250909_20250910000617.CSV (data: 20250909) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250910_20250911000623.CSV (data: 20250910) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250911_20250912000607.CSV (data: 20250911) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250912_20250913000613.CSV (data: 20250912) — já inserido ou fora do ano alvo.
IGNORADO: CAX019_TOQUE_BANCA_D_20250915_20250916000609.CSV (data: 20250915) — já inserido ou fora do ano alvo.
I

recolhas

In [ ]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['ctt2025']
    return db

# Criar coleção (opcional — Mongo cria automaticamente ao inserir)
def criar_colecao(nome_colecao):
    db = conecta_db()
    if nome_colecao not in db.list_collection_names():
        db.create_collection(nome_colecao)
        print(f"Coleção '{nome_colecao}' criada.")
    else:
        print(f"Coleção '{nome_colecao}' já existe.")

# Inserir documento (linha)
def inserir_documento(nome_colecao, documento):
    db = conecta_db()
    colecao = db[nome_colecao]
    try:
        colecao.insert_one(documento)
        print("Documento inserido com sucesso.")
    except PyMongoError as error:
        print(f"Erro ao inserir documento: {error}")

# Criar coleção equivalente à tabela Recolhas_2025
criar_colecao("recolhas_2025")

# Exemplo de documento (linha)
documento_exemplo = {
    "data_ficheiro": "2025.04.24",
    "Código Orgânico": "ORG001",
    "Descrição": "Serviço diário",
    "Dia": "Quinta-feira",
    "Giro": "G1",
    "Tipo": "Normal",
    "Ponto Recolha": "Ponto A",
    "Nome Cliente": "Cliente XPTO",
    "Morada de recolha": "Rua das Flores, 10",
    "Código postal recolha": "1000-001",
    "Qntd de Objetos Recolhidos": "12",
    "Estado": "Concluído",
    "Hora Início": "08:00",
    "Hora Fim": "10:00",
    "Hora Real": "09:35"
}

# Inserir esse documento na coleção
#inserir_documento("Recolhas_2025", documento_exemplo)


Coleção 'recolhas_2025' já existe.


In [ ]:
from pymongo import MongoClient
import numpy as np
import re

def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Conectar à base de dados
db = conecta_db()
colecao = db['recolhas_2025']

# Obter todas as datas distintas do campo data_ficheiro
datas = colecao.distinct("data_ficheiro")

data_validation = []

# Expressão regular para datas no formato YYYY.MM.DD
regex_data = re.compile(r"^(\d{4})\.(\d{2})\.(\d{2})$")



for data in datas:
    if isinstance(data, str):
        match = regex_data.match(data.strip())
        if match:
            ano, mes, dia = match.groups()
            data_formatada = ano + mes.zfill(2) + dia.zfill(2)
            data_validation.append(data_formatada)
        else:
            print(f"Formato inválido ignorado: {data}")

# Eliminar duplicados, ordenar e converter para numpy array
data_validation = np.array(sorted(set(data_validation)))
print(data_validation)


['20250823' '20250824' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250830' '20250831' '20250901' '20250902' '20250903'
 '20250904' '20250905' '20250906' '20250907' '20250908' '20250909'
 '20250910' '20250911' '20250912' '20250913' '20250914' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250920' '20250921']


In [ ]:
import pandas as pd
import numpy as np
import io
import time
from ftplib import FTP
from pymongo import MongoClient
import re

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Obter datas já existentes em MongoDB (Recolhas_2025)
def obter_datas_existentes():
    db = conecta_db()
    colecao = db["recolhas_2025"]
    datas = colecao.distinct("data_ficheiro")

    regex_data = re.compile(r"^(\d{4})\.(\d{2})\.(\d{2})$")


    datas_formatadas = []

    for data in datas:
        if isinstance(data, str):
            match = regex_data.match(data.strip())
            if match:
                ano, mes, dia = match.groups()
                data_formatada = ano + mes + dia
                datas_formatadas.append(data_formatada)
            else:
                print(f"Formato inválido ignorado: {data}")

    return set(datas_formatadas)


# Processamento de cada linha do ficheiro
def process_data(row):
    try:
        return row.decode('Latin-1').strip()
    except UnicodeDecodeError:
        return row.decode('utf-8').strip()

# Nomes das colunas
columns = ['"Código Orgânico"', '"Descrição"', '"Dia"', '"Giro"', '"Tipo"', '"Ponto Recolha"', '"Nome Cliente"',
           '"Morada de recolha"', '"Código postal recolha"', '"Qntd de Objetos Recolhidos"', '"Estado"', '"Hora Início"',
           '"Hora Fim"', '"Hora Real"']

# Obter lista de ficheiros do FTP
ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')
ftp.cwd('/OP/Rotas/Banca_Recolhas')
files = ftp.nlst()
ftp.quit()

# Datas já presentes no MongoDB
data_validation = obter_datas_existentes()

# Processar ficheiros
for file in files:
    start_time = time.time()
    if file.startswith("R"):
        date_str_s = file[19:27].replace("/", "")
        ano = file[19:23]

        if date_str_s not in data_validation and ano == "2025":
            # Nova ligação FTP para o ficheiro
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]
            ftp.quit()

            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]

            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')
            decoded_data = csv_data.decode('utf-8-sig')

            if decoded_data.strip():
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', names=columns, on_bad_lines='skip', encoding="Latin-1")
                df = df.iloc[1:]  # remover header duplicado
                df['data_ficheiro'] = f"{ano}.{date_str_s[4:6]}.{date_str_s[6:]}"
                df = df.where(pd.notna(df), None)

                db = conecta_db()
                colecao = db['recolhas_2025']

                try:
                    documentos = df.to_dict(orient='records')
                    if documentos:
                        colecao.insert_many(documentos)
                        print(f"Ficheiro '{file}' inserido com sucesso ({len(documentos)} registos). Tempo: {time.time() - start_time:.2f}s")
                except Exception as e:
                    print(f"Erro ao inserir dados do ficheiro '{file}': {e}")
        else:
            print(f"Ficheiro '{file}' ignorado (já existente ou fora do ano alvo).")


Ficheiro 'RECSG4366_RECOLHAS_20250903_20250904010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250904_20250905010501.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250905_20250906010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250906_20250907045637.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250907_20250908010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250908_20250909010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250909_20250910010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250910_20250911010501.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250911_20250912010500.CSV' ignorado (já existente ou fora do ano alvo).
Ficheiro 'RECSG4366_RECOLHAS_20250912_20250913010501.CSV' ignorado (já ex

In [ ]:
# rede base

In [ ]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Criar coleção (opcional — Mongo cria automaticamente ao inserir)
def criar_colecao(nome_colecao):
    db = conecta_db()
    if nome_colecao not in db.list_collection_names():
        db.create_collection(nome_colecao)
        print(f"Coleção '{nome_colecao}' criada.")
    else:
        print(f"Coleção '{nome_colecao}' já existe.")

# Inserir documento na coleção
def inserir_documento(nome_colecao, documento):
    db = conecta_db()
    colecao = db[nome_colecao]

    # Verificação para evitar duplicados com base na DATA_CRIACAO
    data_chave = documento.get("data_criacao")
    if data_chave:
        existe = colecao.find_one({"data_criacao": data_chave})
        if existe:
            print(f"Documento com DATA_CRIACAO '{data_chave}' já existe. Ignorado.")
            return

    try:
        colecao.insert_one(documento)
        print("Documento inserido com sucesso.")
    except PyMongoError as error:
        print(f"Erro ao inserir documento: {error}")

# Criar coleção
criar_colecao("rede_base_2025")

# Exemplo de documento
documento_exemplo = {
    "DATA_CRIACAO": "2025.04.24",
    "CENTRO": "Centro Norte",
    "Giro": "G45",
    "LOPTICA": "L1234",
    "JANELA_HORARIA": "08:00-10:00",
    "NOME": "Loja XPTO",
    "MORADA": "Rua Exemplo, 100",
    "CP": "4000-123",
    "LOCALIDADE": "Porto",
    "COD_T_EVEN": "EV123",
    "DATA_EVENTO": "2025.04.23",
    "LATITUDE": "41.14961",
    "LONGITUDE": "-8.61099",
    "NOME_REM": "Remetente ABC",
    "COD_PAIS_ORIGEM": "PT"
}

# Inserir documento
#inserir_documento("REDE_BASE_2025", documento_exemplo)


Coleção 'rede_base_2025' já existe.


In [ ]:
from pymongo import MongoClient
import numpy as np
import re

def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Conectar à base de dados
db = conecta_db()
colecao = db['rede_base_2025']

# Obter todas as datas distintas do campo DATA_CRIACAO
datas = colecao.distinct("data_criacao")

data_validation = []

# Expressão regular para datas no formato YYYY.MM.DD
regex_data = re.compile(r"^(\d{2})/(\d{2})/(\d{4})$")

for data in datas:
    if isinstance(data, str):
        match = regex_data.match(data.strip())
        if match:
            dia, mes, ano = match.groups()
            data_formatada = ano + mes.zfill(2) + dia.zfill(2)
            data_validation.append(data_formatada)
        else:
            print(f"Formato inválido ignorado: {data}")

# Eliminar duplicados, ordenar e converter para numpy array
data_validation = np.array(sorted(set(data_validation)))
print(data_validation)


['20250823' '20250824' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250830' '20250831' '20250901' '20250902' '20250903'
 '20250904' '20250905' '20250906' '20250907' '20250908' '20250909'
 '20250910' '20250911' '20250912' '20250913' '20250914' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250920' '20250921']


In [ ]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
from pymongo import MongoClient
import time
import re

# Conexão MongoDB
def conecta_db():
    client = MongoClient('mongodb://localhost:27017/')
    return client['ctt2025']

# Verifica datas já existentes na coleção REDE_BASE_2025
def obter_datas_existentes():
    db = conecta_db()
    colecao = db["rede_base_2025"]
    datas = colecao.distinct("data_criacao")

    regex_data = re.compile(r"^(\d{2})/(\d{2})/(\d{4})$")
    datas_formatadas = []

    for data in datas:
        if isinstance(data, str):
            match = regex_data.match(data.strip())
            if match:
                dia, mes, ano = match.groups()
                data_formatada = ano + mes.zfill(2) + dia.zfill(2)
                datas_formatadas.append(data_formatada)
    
    return set(datas_formatadas)

# Processa linha
def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError:
        decoded_line = row.decode('utf-8')

    if len(decoded_line.split(';')) == 13:
        return decoded_line.strip()
    else:
        print(f"Ignorando linha com colunas incorretas: {decoded_line}")
        return None

# Ligar FTP e obter ficheiros
ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')
ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')
files = ftp.nlst()
ftp.quit()

# Obter datas validadas
data_validation = obter_datas_existentes()

# Processar ficheiros
for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        ano = file[14:18]
        mes = file[18:20]

        if date_str_s not in data_validation and ano == "2025" and mes >= "01":
            print(f"A processar: {date_str_s}")

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)

            try:
                df = pd.read_csv(file_data, sep=';', on_bad_lines='skip', encoding="Latin-1")
            except pd.errors.ParserError as pe:
                print(f"Erro ao ler CSV no ficheiro '{file}': {pe}")
                ftp.quit()
                continue

            ftp.quit()

            if df.empty:
                print(f"Ficheiro '{file}' vazio ou com linhas inválidas. Ignorado.")
                continue

            df = df.where(pd.notna(df), None)
            df["data_criacao"] = f"{date_str_s[6:]}/{mes}/{ano}"


            db = conecta_db()
            colecao = db['rede_base_2025']

            try:
                documentos = df.to_dict(orient='records')
                if documentos:
                    colecao.insert_many(documentos)
                    print(f"Ficheiro '{file}' inserido com sucesso ({len(documentos)} registos). Tempo: {time.time() - start_time:.2f}s")
            except Exception as e:
                print(f"Erro ao inserir dados do ficheiro '{file}': {e}")
        else:
            print(f"Ficheiro '{file}' ignorado (já inserido ou data inválida).")


Ficheiro 'EVRBP4348_ENV_20250903_20250904010512.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250904_20250905010506.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250905_20250906010507.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250906_20250907045626.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250907_20250908010515.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250908_20250909010504.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250909_20250910010505.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250910_20250911010508.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250911_20250912010502.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250912_20250913010520.CSV' ignorado (já inserido ou data inválida).
Ficheiro 'EVRBP4348_ENV_20250913_20250914010502.CSV' ignorad